In [ ]:
import sys, os
if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash

    !pip install -q gymnasium

    !touch .setup_complete

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

### Let's make a TRPO!

In this notebook we will write the code of the one Trust Region Policy Optimization.
As usually, it contains a few different parts which we are going to reproduce.



In [4]:
from typing import Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import gymnasium as gym

env = gym.make("Acrobot-v1", render_mode="rgb_array")
env.reset()
observation_shape = env.observation_space.shape
n_actions = env.action_space.n

print("Observation Space", env.observation_space)
print("Action Space", env.action_space)

In [ ]:
plt.imshow(env.render())

### Step 1: Defining a network

With all it's complexity, at it's core TRPO is yet another policy gradient method.

This essentially means we're actually training a stochastic policy $\pi_\theta \left( a \middle| s \right)$.

And yes, it's gonna be a neural network. So let's start by defining one.

In [38]:
class TRPOAgent(nn.Module):
    def __init__(self, state_shape: Tuple[int], n_actions: int):
        '''
        Here you should define your model
        You should have LOG-PROBABILITIES as output because you will need it to compute loss
        We recommend that you start simple:
        use 1-2 hidden layers with 100-500 units and relu for the first try
        '''
        super().__init__()

        assert isinstance(state_shape, tuple)
        assert len(state_shape) == 1
        input_dim = state_shape[0]

        # Prepare your model here.
        self.model = #YOUR CODE HERE

    def forward(self, states: torch.Tensor):
        """
        takes agent's observation, returns logits
        :param state_t: a batch of states, shape = [batch_size, state_shape]
        """

        # Use your network to compute logits for the given states.
        logits = #YOUR CODE HERE

        return logits

    def get_log_probs(self, states: torch.Tensor):
        '''
        Log-probs for training
        '''
        return #YOUR CODE HERE

    def get_probs(self, states: torch.Tensor):
        '''
        Probs for interaction
        '''
        return #YOUR CODE HERE

    def act(self, obs: np.ndarray, sample: bool = True):
        '''
        Samples action from policy distribution (sample = True) or takes most likely action (sample = False)
        :param: obs - single observation vector
        :param sample: if True, samples from \pi, otherwise takes most likely action
        :returns: action (single integer) and probabilities for all actions
        '''

        with torch.no_grad():
            probs = self.get_probs(torch.tensor(obs[np.newaxis], dtype=torch.float32)).numpy()
        if sample:
            action = int(np.random.choice(n_actions, p=probs[0]))
        else:
            action = int(np.argmax(probs))

        return action, probs[0]


agent = TRPOAgent(observation_shape, n_actions)

In [ ]:
# Check if log-probabilities satisfies all the requirements
log_probs = agent.get_log_probs(torch.tensor(env.reset()[0][np.newaxis], dtype=torch.float32))
assert (
    isinstance(log_probs, torch.Tensor) and
    log_probs.requires_grad
), "log_probs must be a torch.Tensor with grad"
assert log_probs.shape == (1, n_actions)
sums = torch.exp(log_probs).sum(dim=1)
assert torch.allclose(sums, torch.ones_like(sums))

# Demo use
print("sampled:", [agent.act(env.reset()[0]) for _ in range(5)])
print("greedy:", [agent.act(env.reset()[0], sample=False) for _ in range(5)])

#### Flat parameters operations

We are going to use it

In [10]:
def get_flat_params_from(model):
    params = [torch.ravel(param.detach()) for param in model.parameters()]
    flat_params = torch.cat(params)
    return flat_params


def set_flat_params_to(model, flat_params):
    prev_ind = 0
    for param in model.parameters():
        flat_size = int(np.prod(list(param.shape)))
        param.data.copy_(
            flat_params[prev_ind:prev_ind + flat_size].reshape(param.shape)
        )
        prev_ind += flat_size

Compute cumulative reward just like you did in vanilla REINFORCE

In [11]:
def get_cumulative_returns(rewards, gamma=1):
    """
    Computes cumulative discounted rewards given immediate rewards
    G_i = r_i + gamma*r_{i+1} + gamma^2*r_{i+2} + ...
    Also known as G(s,a).
    """
    G = 0
    returns = []
    for r in reversed(rewards):
        G = r + gamma * G
        returns.append(G)
    return np.array(returns[::-1])

In [ ]:
# simple demo on rewards [0,0,1,0,0,1]
get_cumulative_returns([0, 0, 1, 0, 0, 1], gamma=0.9)

**Rollout**

In [15]:
def rollout(env, agent, max_pathlength=2500, n_timesteps=50000):
    """
    Generate rollouts for training.
    :param: env - environment in which we will make actions to generate rollouts.
    :param: act - the function that can return policy and action given observation.
    :param: max_pathlength - maximum size of one path that we generate.
    :param: n_timesteps - total sum of sizes of all pathes we generate.
    """
    paths = []

    total_timesteps = 0
    while total_timesteps < n_timesteps:
        obervations, actions, rewards, action_probs = [], [], [], []
        obervation = env.reset()[0]
        for _ in range(max_pathlength):
            action, policy = agent.act(obervation)
            obervations.append(obervation)
            actions.append(action)
            action_probs.append(policy)
            obervation, reward, terminated, truncated, _ = env.step(action)
            rewards.append(reward)
            total_timesteps += 1
            if terminated or truncated or total_timesteps >= n_timesteps:
                path = {
                    "observations": np.array(obervations),
                    "policy": np.array(action_probs),
                    "actions": np.array(actions),
                    "rewards": np.array(rewards),
                    "cumulative_returns": get_cumulative_returns(rewards),
                }
                paths.append(path)
                break
    return paths

In [21]:
paths = rollout(env, agent, max_pathlength=5, n_timesteps=100)

assert (paths[0]['policy'].shape == (5, n_actions))
assert (paths[0]['cumulative_returns'].shape == (5,))
assert (paths[0]['rewards'].shape == (5,))
assert (paths[0]['observations'].shape == (5,) + observation_shape)
assert (paths[0]['actions'].shape == (5,))

print("It's ok")

It's ok


### Step 3: Auxiliary functions

Now let's define the loss functions and something else for actual TRPO training.

The surrogate reward should be:
$$J_{surr}= {1 \over N} \sum\limits_{i=1}^N \frac{\pi_{\theta}(s_i, a_i)}{\pi_{\theta_{old}}(s_i, a_i)}A_{\theta_{old}(s_i, a_i)}$$

For simplicity, in this assignment we are going to use cumulative rewards instead of advantage:
$$J'_{surr}= {1 \over N} \sum\limits_{i=1}^N \frac{\pi_{\theta}(s_i, a_i)}{\pi_{\theta_{old}}(s_i, a_i)}G_{\theta_{old}(s_i, a_i)}$$

Since we want to maximize the reward, we are going to minimize the corresponding surrogate loss:
$$ L_{surr} = - J'_{surr} $$


In [22]:
def get_loss(agent, observations, actions, cumulative_returns, old_probs):
    """
    Computes TRPO objective
    :param: observations - batch of observations [timesteps x state_shape]
    :param: actions - batch of actions [timesteps]
    :param: cumulative_returns - batch of cumulative returns [timesteps]
    :param: old_probs - batch of probabilities computed by old network [timesteps x num_actions]
    :returns: scalar value of the objective function
    """
    batch_size = observations.shape[0]
    new_probs = agent.get_probs(observations)

    new_probs_for_actions = new_probs[torch.arange(batch_size), actions]
    old_probs_for_actions = old_probs[torch.arange(batch_size), actions]

    # Compute surrogate loss, aka importance-sampled policy gradient
    loss = #YOUR CODE HERE

    assert loss.ndim == 0
    return loss

We can ascend these gradients as long as our $\pi_\theta(a|s)$ satisfies the constraint
$$\mathbb{E}_{s,\pi_{\theta_{t}}} \Big[ \operatorname{KL} \left( \pi_{\theta_{t}} (s) \:\|\: \pi_{\theta_{t+1}} (s) \right) \Big] < \alpha$$


where

$$\operatorname{KL} \left( p \| q \right) = \mathbb{E}_p \log \left( \frac p q \right)$$

In [23]:
def get_kl(agent, observations, actions, cumulative_returns, old_probs):
    """
    Computes KL-divergence between network policy and old policy
    :param: observations - batch of observations [timesteps x state_shape]
    :param: actions - batch of actions [timesteps]
    :param: cumulative_returns - batch of cumulative returns [timesteps] (we don't need it actually)
    :param: old_probs - batch of probabilities computed by old network [timesteps x num_actions]
    :returns: scalar value of the KL-divergence
    """
    batch_size = observations.shape[0]
    new_log_probs = agent.get_log_probs(observations)

    # Compute Kullback-Leibler divergence (see formula above).
    # Note: you need to sum KL and entropy over all actions, not just the ones agent took.
    # You will also need to compute max KL over all timesteps.
    old_log_probs = torch.log(old_probs + 1e-10)

    kl = #YOUR CODE HERE

    assert kl.ndim == 0
    assert (kl > -0.0001).all() and (kl < 10000).all()
    return kl

In [29]:
def get_entropy(agent, observations):
    """
    Computes entropy of the network policy
    :param: observations - batch of observations
    :returns: scalar value of the entropy
    """

    observations = torch.tensor(observations, dtype=torch.float32)

    log_probs = agent.get_log_probs(observations)
    probs = agent.get_probs(observations)

    entropy = #YOUR CODE HERE

    assert entropy.ndim == 0
    return entropy

**Linear search**

TRPO in its core involves ascending surrogate policy gradient constrained by KL divergence.

In order to enforce this constraint, we're gonna use linesearch. You can find out more about it [here](https://en.wikipedia.org/wiki/Linear_search)

In [30]:
def linesearch(f, x: torch.Tensor, fullstep: torch.Tensor, max_kl: float, max_backtracks: int = 10, backtrack_coef: float = 0.5):
    """
    Linesearch finds the best parameters of neural networks in the direction of fullstep contrainted by KL divergence.
    :param: f - function that returns loss, kl and arbitrary third component.
    :param: x - old parameters of neural network.
    :param: fullstep - direction in which we make search.
    :param: max_kl - constraint of KL divergence.
    :returns:
    """
    loss, _, = f(x)
    for stepfrac in backtrack_coef**np.arange(max_backtracks):
        xnew = x + stepfrac * fullstep
        new_loss, kl = f(xnew)
        if kl <= max_kl and new_loss < loss:
            x = xnew
            loss = new_loss
    return x

**Conjugate gradients**

Since TRPO includes contrainted optimization, we will need to solve $A x = b$ using conjugate gradients.

In general, CG is an algorithm that solves $A x = b$ where $A$ is positive-defined. $A$ is the Hessian matrix so $A$ is positive-defined. You can find out more about CG [here](https://en.wikipedia.org/wiki/Conjugate_gradient_method).

In [31]:
def conjugate_gradient(f_Ax, b, cg_iters=10, residual_tol=1e-10):
    """
    This method solves system of equation Ax=b using an iterative method called conjugate gradients
    :f_Ax: function that returns Ax
    :b: targets for Ax
    :cg_iters: how many iterations this method should do
    :residual_tol: epsilon for stability
    """
    p = b.clone()
    r = b.clone()
    x = torch.zeros_like(b)
    rdotr = torch.sum(r*r)
    for i in range(cg_iters):
        z = f_Ax(p)
        v = rdotr / (torch.sum(p*z) + 1e-8)
        x += v * p
        r -= v * z
        newrdotr = torch.sum(r*r)
        mu = newrdotr / (rdotr + 1e-8)
        p = r + mu * p
        rdotr = newrdotr
        if rdotr < residual_tol:
            break
    return x

In [ ]:
# This code validates conjugate gradients
A = np.random.rand(8, 8)
A = A.T @ A


def f_Ax(x):
    return torch.ravel(torch.tensor(A, dtype=torch.float32) @ x.reshape(-1, 1))


b = np.random.rand(8)
w = (np.linalg.inv(A.T @ A) @ A.T @ b.reshape(-1, 1)).reshape(-1)

print(w)
print(conjugate_gradient(f_Ax, torch.tensor(b, dtype=torch.float32)).numpy())

### Step 4: training
In this section we construct the whole update step function.

In [33]:
def update_step(agent, observations, actions, cumulative_returns, old_probs, max_kl):
    """
    This function does the TRPO update step
    :param: observations - batch of observations
    :param: actions - batch of actions
    :param: cumulative_returns - batch of cumulative returns
    :param: old_probs - batch of probabilities computed by old network
    :param: max_kl - controls how big KL divergence may be between old and new policy every step.
    :returns: KL between new and old policies and the value of the loss function.
    """

    # Here we prepare the information
    observations = torch.tensor(observations, dtype=torch.float32)
    actions = torch.tensor(actions, dtype=torch.int64)
    cumulative_returns = torch.tensor(cumulative_returns, dtype=torch.float32)
    old_probs = torch.tensor(old_probs, dtype=torch.float32)

    # Here we compute gradient of the loss function
    loss = get_loss(agent, observations, actions, cumulative_returns, old_probs)
    grads = torch.autograd.grad(loss, agent.parameters())
    loss_grad = torch.cat([torch.ravel(grad.detach()) for grad in grads])

    def Fvp(v):
        # Here we compute Fx to solve Fx = g using conjugate gradients
        # We actually do here a couple of tricks to compute it efficiently

        kl = get_kl(agent, observations, actions, cumulative_returns, old_probs)

        grads = torch.autograd.grad(kl, agent.parameters(), create_graph=True)
        flat_grad_kl = torch.cat([grad.reshape(-1) for grad in grads])

        kl_v = (flat_grad_kl * v).sum()
        grads = torch.autograd.grad(kl_v, agent.parameters())
        flat_grad_grad_kl = torch.cat([torch.ravel(grad) for grad in grads]).detach()

        return flat_grad_grad_kl + v * 0.1

    # Here we solve Fx = g system using conjugate gradients
    stepdir = conjugate_gradient(Fvp, -loss_grad, 10)

    # Here we compute the initial vector to do linear search
    shs = 0.5 * (stepdir * Fvp(stepdir)).sum(0, keepdim=True)

    lm = torch.sqrt(shs / max_kl)
    fullstep = stepdir / lm[0]

    neggdotstepdir = (-loss_grad * stepdir).sum(0, keepdim=True)

    # Here we get the start point
    prev_params = get_flat_params_from(agent)

    def get_loss_kl(params):
        # Helper for linear search
        set_flat_params_to(agent, params)
        return [
            get_loss(agent, observations, actions, cumulative_returns, old_probs),
            get_kl(agent, observations, actions, cumulative_returns, old_probs),
        ]

    # Here we find our new parameters
    new_params = linesearch(get_loss_kl, prev_params, fullstep, max_kl)

    # And we set it to our network
    set_flat_params_to(agent, new_params)

    return get_loss_kl(new_params)

### Step 5: Main TRPO loop

Here we will train our network!

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt

agent = TRPOAgent(observation_shape, n_actions)
# TRPO hyperparameter; controls how big KL divergence may be between the old and the new policy at every step.
max_kl = 0.01
numeptotal = 0  # Number of episodes we have completed so far.


reward_history = []
kl_history = []
entropy_history = []
loss_history = []

for i in range(100):

    # Generating paths.
    paths = rollout(env, agent)

    # Updating policy.
    observations = np.concatenate([path["observations"] for path in paths])
    actions = np.concatenate([path["actions"] for path in paths])
    returns = np.concatenate([path["cumulative_returns"] for path in paths])
    old_probs = np.concatenate([path["policy"] for path in paths])

    loss, kl = update_step(agent, observations, actions, returns, old_probs, max_kl)

    # Report current progress
    episode_rewards = np.array([path["rewards"].sum() for path in paths])

    stats = {}
    numeptotal += len(episode_rewards)

    reward_history.append(episode_rewards.mean())
    kl_history.append(kl.detach().item())
    entropy_history.append(get_entropy(agent, observations).detach().item())
    loss_history.append(loss.detach().item())

    clear_output(True)
    plt.figure(figsize=[16, 9])

    plt.subplot(2, 2, 1)
    plt.title("Mean reward")
    plt.plot(reward_history)
    plt.grid()

    plt.subplot(2, 2, 2)
    plt.title("Surrogate loss")
    plt.plot(loss_history)
    plt.grid()

    plt.subplot(2, 2, 3)
    plt.title("Enropy")
    plt.plot(entropy_history)
    plt.grid()

    plt.subplot(2, 2, 4)
    plt.title("KL")
    plt.plot(kl_history)
    plt.grid()

    plt.show()

In [ ]:
# record sessions
from gymnasium.wrappers import RecordVideo

with RecordVideo(env=gym.make("Acrobot-v1", render_mode="rgb_array"), video_folder='./videos',
                 episode_trigger = lambda episode_number: True) as env_monitor:
    rollout(env_monitor, agent, max_pathlength=100, n_timesteps=100)
env.close()

In [ ]:
# Show video. This may not work in some setups. If it doesn't
# work for you, you can download the videos and view them locally.

from pathlib import Path
from base64 import b64encode
from IPython.display import HTML

video_paths = sorted([s for s in Path('videos').iterdir() if s.suffix == '.mp4'])
video_path = video_paths[1]  # You can also try other indices

if 'google.colab' in sys.modules:
    # https://stackoverflow.com/a/57378660/1214547
    with video_path.open('rb') as fp:
        mp4 = fp.read()
    data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
else:
    data_url = str(video_path)

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(data_url))